# Interactive Functions 

In [6]:
#! pip install plotnine

In [7]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [8]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [9]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

In [10]:
df.sample(5)

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
18378,CML,5184(038),Tehachapi,2021-02-25,68080.00,0.00,76900.00,Authorized,5184,9,KER200505,Pinon Street From Brandon Lane East To Denniso...,Pave An Unpaved Portion Of Road And Install A ...,1,KCOG,2021-01-01
10136,CML,5398(007),South Lake Tahoe,2015-01-06,0.00,0.00,-3098.60,Authorized,5398,3,GROUP 1,In City Of South Lake Tahoe: Pioneer Trail Fro...,"S/w, Ada Impr, Bike Lanes, X-walks, Lighting",4,TMPO,2015-01-01
16913,ACST-ER,32L0(040),Pismo Beach,2020-07-17,0.00,2278121.00,5109932.00,Authorized,5289,5,NaN,On Ocean Blvd (btwn Cuyama & Morro) In Pismo B...,Bluff Face Stabilization,3,SLAC,2020-01-01
8730,HSIP,5068(051),Redding,2015-07-01,100800.00,0.00,83800.00,Authorized,5068,2,GP-HSIP,On Victor Avenue Between Enterprise Park And C...,"Road Diet, Median Islands, Ped Path, Lighting ...",2,STPA,2015-01-01
15318,STP,5124(035),Mountain View,2019-12-03,1136000.00,0.00,1630880.00,Authorized,5124,4,SCL170040,West Middlefield Road From Rengstroff Avenue T...,Resurface Roadway And Reconstruct Median Island,1,MTC,2019-01-01


## Full Function for Prefix

In [11]:
    @interact
    
    def dla_get_prefix(place=df.agency.sort_values().unique().tolist()):
        
        agencies = df[df.agency==place]
    
        prefix_count_n = agencies >> count(_.prefix)
    
        display(Markdown(f"**Summary Statistics for {place}**"))
        display(Markdown(f"The number of obligations {place} has is {len(agencies)}"))
        
        display(Markdown(f"The number of prefix codes {place} uses is {len(prefix_count_n)}"))
        
        
        
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        display(df[df.agency == place][['fed_requested','ac_requested','total_requested']].describe())
        
        display(Markdown(f"**Top Project Types in {place}**"))
        display((df[df.agency == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
         

    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        return ax1

        

interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa…

## Function for results by Prefix

In [12]:
@interact
def prefix_all_agencies_4(prefix=df.prefix.sort_values().unique().tolist()):
    
    
    # graphs 
    prefixes = df[df.prefix== prefix]
    
    prefix_count_num = (prefixes >> count(_.agency) >> arrange(-_.n))
    
    prefix_count = (prefixes >> count(_.agency) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix', options=('6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', 'ASCTP'…

## Function for Type of Work

In [13]:
top50 = (df >> count(_.type_of_work) >> arrange(-_.n)).head(50)

In [14]:
@interact
def findtypeofwork(work=top50.type_of_work.sort_values().unique().tolist()):
    
    display(Markdown(f"**Number of Obligations the top Agencies have in {work}**"))
    # graphs 
    worktypes = df[df.type_of_work== work]
    
    work_count = (worktypes >> count(_.agency) >> arrange(-_.n)).head(20)
    display(work_count)

    ax1 = (work_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Agencies using Type of Work', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1

interactive(children=(Dropdown(description='work', options=('Ac Overlay', 'Ac Overlay (tc)', 'Asphalt Concrete…

## Function to Filter Data

In [15]:
df_query = df[['prefix', 'agency', 'project_no','locode','dist','fed_requested', 'ac_requested', 'total_requested',
              'type_of_work','project_location', 'seq', 'mpo','prepared_y']]

In [16]:
#adding year
@interact
def get_query2(agency=df_query.agency.unique().tolist(),
             prefix=df_query.prefix.unique().tolist(),
             year=df_query.prepared_y.unique().tolist()):
   
    filtering = df_query[(df_query.agency==agency)&(df_query.prefix==prefix)&(df_query.prepared_y==year)]
    
    
    display(Markdown(f"**The number of obligations {agency} has in {prefix} during the year {year} is {len(filtering)}**"))
    
    display(filtering.head())
    
    

interactive(children=(Dropdown(description='agency', options=('Humboldt County', 'Mendocino County', 'Sacramen…